In [163]:
import json
import numpy as np
import pandas as pd
from dateutil import parser
import preprocess_hop_log, merge_hop_tick, encode_brokerid
import os

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

In [164]:
# path_hoplog = '../raw_data/hopLogger/12/'
# path_ticklog = '../raw_data/tickLogger/12/'
# path_mergeddf = '../preprocessed_data/12/'

DIR_NAME = 89

path_hoplog = '../../wowsan/log/hopLogger/' + str(DIR_NAME) + '/'
path_ticklog = '../../wowsan/log/tickLogger/' + str(DIR_NAME) + '/'
path_mergeddf = '../preprocessed_data/' + str(DIR_NAME) + '/'


## Hop Log 전처리

In [165]:
# json 파일을 열고 각 줄을 개별적으로 파싱
# path = '../raw_data/hopLogger/116/'
data = []
with open(path_hoplog + '50002.json', 'r') as file:
    for line in file:
        data.append(json.loads(line))

# 데이터를 DataFrame으로 변환
hop_df_raw = pd.DataFrame(data)
type(hop_df_raw)

pandas.core.frame.DataFrame

In [166]:
hop_df_raw

,Node,PerformanceInfo,level,msg,time
0,localhost:50002,"[{'BrokerId': 'localhost:50003', 'Cpu': '99.396370', 'Memory': '23515136', 'QueueLength': '3', 'QueueTime': '0.903563', 'AverageQueueTime': '', 'ServiceTime': '0.000000', 'AverageServiceTime': '', 'ResponseTime': '0.903563', 'InterArrivalTime': '0.001165', 'Throughput': '1.106730', 'AverageThroughput': '', 'Timestamp': '2024-08-03 19:29:34.220976 +0900 KST m=+9.128024101'}, {'BrokerId': 'localhost:50002', 'Cpu': '100.110671', 'Memory': '23736320', 'QueueLength': '3', 'QueueTime': '1.162724', 'AverageQueueTime': '1.162724', 'ServiceTime': '0.000000', 'AverageServiceTime': '0.000000', 'ResponseTime': '1.162724', 'InterArrivalTime': '0.227600', 'Throughput': '0.860050', 'AverageThroughput': '0.860050', 'Timestamp': '2024-08-03 19:29:35.3887074 +0900 KST m=+10.295749401'}]",info,Advertisement(apple > 0.8346767073039755) ca036fac-2b7f-4d21-9e1f-81f1f4dbedb0,2024-08-03 19:29:35
1,localhost:50002,"[{'BrokerId': 'localhost:50001', 'Cpu': '99.684715', 'Memory': '23453696', 'QueueLength': '1', 'QueueTime': '1.161399', 'AverageQueueTime': '', 'ServiceTime': '0.000000', 'AverageServiceTime': '', 'ResponseTime': '1.161399', 'InterArrivalTime': '0.001046', 'Throughput': '0.861030', 'AverageThroughput': '', 'Timestamp': '2024-08-03 19:29:34.4793259 +0900 KST m=+9.390476801'}, {'BrokerId': 'localhost:50002', 'Cpu': '100.019933', 'Memory': '24195072', 'QueueLength': '3', 'QueueTime': '2.070749', 'AverageQueueTime': '1.616736', 'ServiceTime': '1.170726', 'AverageServiceTime': '1.170726', 'ResponseTime': '3.241475', 'InterArrivalTime': '0.284757', 'Throughput': '0.308502', 'AverageThroughput': '0.358749', 'Timestamp': '2024-08-03 19:29:36.5540863 +0900 KST m=+11.461122201'}]",info,Advertisement(apple > 0.5348975819733137) 76ad8890-e29f-41cb-8e1d-7df43ca35e1b,2024-08-03 19:29:36
2,localhost:50002,"[{'BrokerId': 'localhost:50004', 'Cpu': '100.851342', 'Memory': '23453696', 'QueueLength': '0', 'QueueTime': '1.161399', 'AverageQueueTime': '', 'ServiceTime': '0.000000', 'AverageServiceTime': '', 'ResponseTime': '1.161399', 'InterArrivalTime': '0.000000', 'Throughput': '0.861030', 'AverageThroughput': '', 'Timestamp': '2024-08-03 19:29:34.4793259 +0900 KST m=+9.342414701'}, {'BrokerId': 'localhost:50002', 'Cpu': '100.023972', 'Memory': '24453120', 'QueueLength': '2', 'QueueTime': '3.224251', 'AverageQueueTime': '2.152574', 'ServiceTime': '1.162372', 'AverageServiceTime': '1.166549', 'ResponseTime': '4.386623', 'InterArrivalTime': '0.284757', 'Throughput': '0.227966', 'AverageThroughput': '0.301284', 'Timestamp': '2024-08-03 19:29:37.7095955 +0900 KST m=+12.616625401'}]",info,Advertisement(apple > 0.5348975819733137) 25a1782f-2262-442b-80cd-c6baedb9f34a,2024-08-03 19:29:37
3,localhost:50002,"[{'BrokerId': 'localhost:50003', 'Cpu': '99.399814', 'Memory': '24133632', 'QueueLength': '2', 'QueueTime': '1.814963', 'AverageQueueTime': '', 'ServiceTime': '0.908564', 'AverageServiceTime': '', 'ResponseTime': '2.723527', 'InterArrivalTime': '0.001165', 'Throughput': '0.367171', 'AverageThroughput': '', 'Timestamp': '2024-08-03 19:29:35.132381 +0900 KST m=+10.039424301'}, {'BrokerId': 'localhost:50002', 'Cpu': '100.046152', 'Memory': '24461312', 'QueueLength': '1', 'QueueTime': '3.726109', 'AverageQueueTime': '2.545958', 'ServiceTime': '1.150501', 'AverageServiceTime': '1.161200', 'ResponseTime': '4.876610', 'InterArrivalTime': '0.284757', 'Throughput': '0.205060', 'AverageThroughput': '0.269748', 'Timestamp': '2024-08-03 19:29:38.8625104 +0900 KST m=+13.769534301'}]",info,Advertisement(apple > 0.5348975819733137) 9948d2fd-d7a6-4671-bfde-4677809e5177,2024-08-03 19:29:38
4,localhost:50002,"[{'BrokerId': 'localhost:50005', 'Cpu': '99.684467', 'Memory': '22827008', 'QueueLength': '0', 'QueueTime': '1.161911', 'AverageQueueTime': '', 'ServiceTime': '0.000000', 'AverageServiceTime': '', 'ResponseTime': '1.161911', 'InterArrivalTime': '0.000000', 'Throughput': '0.860651', 'AverageThroughput': '', 'Timestamp': '20

In [167]:
hop_df = preprocess_hop_log.parse_performance_info(hop_df_raw)

In [168]:
hop_df

,time,Node,level,msg,HopCount,QueueTime_0,Throughput_0,InterArrivalTime_0,Timestamp_0,QueueLength_0,Cpu_0,ServiceTime_0,BrokerId_0,Memory_0,AverageQueueTime_0,AverageThroughput_0,AverageServiceTime_0,ResponseTime_0,QueueTime_1,Throughput_1,InterArrivalTime_1,Timestamp_1,QueueLength_1,Cpu_1,ServiceTime_1,BrokerId_1,Memory_1,AverageQueueTime_1,AverageThroughput_1,AverageServiceTime_1,ResponseTime_1,QueueTime_2,Throughput_2,InterArrivalTime_2,Timestamp_2,QueueLength_2,Cpu_2,ServiceTime_2,BrokerId_2,Memory_2,AverageQueueTime_2,AverageThroughput_2,AverageServiceTime_2,ResponseTime_2
0,2024-08-03 19:29:35,localhost:50002,info,Advertisement(apple > 0.8346767073039755) ca036fac-2b7f-4d21-9e1f-81f1f4dbedb0,2,1.162724,0.860050,0.227600,2024-08-03 19:29:35.3887074 +0900 KST m=+10.295749401,3,100.110671,0.000000,localhost:50002,23736320,1.162724,0.860050,0.000000,1.162724,0.903563,1.106730,0.001165,2024-08-03 19:29:34.220976 +0900 KST m=+9.128024101,3,99.396370,0.000000,localhost:50003,23515136,,,,0.903563,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-08-03 19:29:36,localhost:50002,info,Advertisement(apple > 0.5348975819733137) 76ad8890-e29f-41cb-8e1d-7df43ca35e1b,2,2.070749,0.308502,0.284757,2024-08-03 19:29:36.5540863 +0900 KST m=+11.461122201,3,100.019933,1.170726,localhost:50002,24195072,1.616736,0.358749,1.170726,3.241475,1.161399,0.861030,0.001046,2024-08-03 19:29:34.4793259 +0900 KST m=+9.390476801,1,99.684715,0.000000,localhost:50001,23453696,,,,1.161399,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-08-03 19:29:37,localhost:50002,info,Advertisement(apple > 0.5348975819733137) 25a1782f-2262-442b-80cd-c6baedb9f34a,2,3.224251,0.227966,0.284757,2024-08-03 19:29:37.7095955 +0900 KST m=+12.616625401,2,100.023972,1.162372,localhost:50002,24453120,2.152574,0.301284,1.166549,4.386623,1.161399,0.861030,0.000000,2024-08-03 19:29:34.4793259 +0900 KST m=+9.342414701,0,100.851342,0.000000,localhost:50004,23453696,,,,1.161399,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-08-03 19:29:38,localhost:50002,info,Advertisement(apple > 0.5348975819733137) 9948d2fd-d7a6-4671-bfde-4677809e5177,2,3.726109,0.205060,0.284757,2024-08-03 19:29:38.8625104 +0900 KST m=+13.769534301,1,100.046152,1.150501,localhost:50002,24461312,2.545958,0.269748,1.161200,4.876610,1.814963,0.367171,0.001165,2024-08-03 19:29:35.132381 +0900 KST m=+10.039424301,2,99.399814,0.908564,localhost:50003,24133632,,,,2.723527,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2024-08-03 19:29:40,localhost:50002,info,Advertisement(apple > 0.5010479585914438) 4996a6de-461e-4014-94a2-83b7ed36aee1,3,4.365799,0.181202,0.284757,2024-08-03 19:29:40.01559 +0900 KST m=+14.922607901,0,100.063807,1.152909,localhost:50002,24477696,2.909926,0.245757,1.159127,5.518708,1.161434,0.429399,0.519194,2024-08-03 19:29:35.6457672 +0900 KST m=+10.508850001,2,100.808678,1.167401,localhost:50004,24334336,,,,2.328835,1.161911,0.860651,0.000000,2024-08-03 19:29:34.4793259 +0900 KST m=+9.346532001,0,99.684467,0.000000,localhost:50005,22827008,,,,1.161911
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6185,2024-08-03 21:29:19,localhost:50002,info,Advertisement(apple > 0.7830639144609146) f012e3a0-1ba8-450f-adb7-54a06446646d,2,4.847540,0.166560,1.160724,2024-08-03 21:29:19.4688354 +0900 KST m=+7194.353995001,3,100.333072,1.156288,localhost:50002,42545152,12.598271,0.072677,1.161250,6.003828,1508.480024,0.000662,0.914329,2024-08-03 21:29:14.6182864 +0900 KST m=+7189.459496301,1662,100.194775,1.156824,localhost:50004,45146112,,,,1509.636848,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6186,2024-08-03 21:29:20,localhost:50002,info,Advertisement(apple > 0.9539755063328923) 569e0625-c121-42b0-9ee4-5c33d6978384,2,5.960860,0.140496,1.160930,2024-08-03 21:29:20.6261676 +0900 KST m=+7195.511325201,3,100.333004,1.156787,localhost:50002,42545152,12.59

### BrokerId_n 컬럼 인코딩

In [169]:
hop_df = encode_brokerid.encode_brokerid(hop_df)

In [170]:
hop_df

,time,Node,level,msg,HopCount,QueueTime_0,Throughput_0,InterArrivalTime_0,Timestamp_0,QueueLength_0,Cpu_0,ServiceTime_0,BrokerId_0,Memory_0,AverageQueueTime_0,AverageThroughput_0,AverageServiceTime_0,ResponseTime_0,QueueTime_1,Throughput_1,InterArrivalTime_1,Timestamp_1,QueueLength_1,Cpu_1,ServiceTime_1,BrokerId_1,Memory_1,AverageQueueTime_1,AverageThroughput_1,AverageServiceTime_1,ResponseTime_1,QueueTime_2,Throughput_2,InterArrivalTime_2,Timestamp_2,QueueLength_2,Cpu_2,ServiceTime_2,BrokerId_2,Memory_2,AverageQueueTime_2,AverageThroughput_2,AverageServiceTime_2,ResponseTime_2
0,2024-08-03 19:29:35,localhost:50002,info,Advertisement(apple > 0.8346767073039755) ca036fac-2b7f-4d21-9e1f-81f1f4dbedb0,2,1.162724,0.860050,0.227600,2024-08-03 19:29:35.3887074 +0900 KST m=+10.295749401,3,100.110671,0.000000,2,23736320,1.162724,0.860050,0.000000,1.162724,0.903563,1.106730,0.001165,2024-08-03 19:29:34.220976 +0900 KST m=+9.128024101,3,99.396370,0.000000,3,23515136,,,,0.903563,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-08-03 19:29:36,localhost:50002,info,Advertisement(apple > 0.5348975819733137) 76ad8890-e29f-41cb-8e1d-7df43ca35e1b,2,2.070749,0.308502,0.284757,2024-08-03 19:29:36.5540863 +0900 KST m=+11.461122201,3,100.019933,1.170726,2,24195072,1.616736,0.358749,1.170726,3.241475,1.161399,0.861030,0.001046,2024-08-03 19:29:34.4793259 +0900 KST m=+9.390476801,1,99.684715,0.000000,1,23453696,,,,1.161399,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-08-03 19:29:37,localhost:50002,info,Advertisement(apple > 0.5348975819733137) 25a1782f-2262-442b-80cd-c6baedb9f34a,2,3.224251,0.227966,0.284757,2024-08-03 19:29:37.7095955 +0900 KST m=+12.616625401,2,100.023972,1.162372,2,24453120,2.152574,0.301284,1.166549,4.386623,1.161399,0.861030,0.000000,2024-08-03 19:29:34.4793259 +0900 KST m=+9.342414701,0,100.851342,0.000000,4,23453696,,,,1.161399,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-08-03 19:29:38,localhost:50002,info,Advertisement(apple > 0.5348975819733137) 9948d2fd-d7a6-4671-bfde-4677809e5177,2,3.726109,0.205060,0.284757,2024-08-03 19:29:38.8625104 +0900 KST m=+13.769534301,1,100.046152,1.150501,2,24461312,2.545958,0.269748,1.161200,4.876610,1.814963,0.367171,0.001165,2024-08-03 19:29:35.132381 +0900 KST m=+10.039424301,2,99.399814,0.908564,3,24133632,,,,2.723527,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2024-08-03 19:29:40,localhost:50002,info,Advertisement(apple > 0.5010479585914438) 4996a6de-461e-4014-94a2-83b7ed36aee1,3,4.365799,0.181202,0.284757,2024-08-03 19:29:40.01559 +0900 KST m=+14.922607901,0,100.063807,1.152909,2,24477696,2.909926,0.245757,1.159127,5.518708,1.161434,0.429399,0.519194,2024-08-03 19:29:35.6457672 +0900 KST m=+10.508850001,2,100.808678,1.167401,4,24334336,,,,2.328835,1.161911,0.860651,0.000000,2024-08-03 19:29:34.4793259 +0900 KST m=+9.346532001,0,99.684467,0.000000,5.0,22827008,,,,1.161911
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6185,2024-08-03 21:29:19,localhost:50002,info,Advertisement(apple > 0.7830639144609146) f012e3a0-1ba8-450f-adb7-54a06446646d,2,4.847540,0.166560,1.160724,2024-08-03 21:29:19.4688354 +0900 KST m=+7194.353995001,3,100.333072,1.156288,2,42545152,12.598271,0.072677,1.161250,6.003828,1508.480024,0.000662,0.914329,2024-08-03 21:29:14.6182864 +0900 KST m=+7189.459496301,1662,100.194775,1.156824,4,45146112,,,,1509.636848,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6186,2024-08-03 21:29:20,localhost:50002,info,Advertisement(apple > 0.9539755063328923) 569e0625-c121-42b0-9ee4-5c33d6978384,2,5.960860,0.140496,1.160930,2024-08-03 21:29:20.6261676 +0900 KST m=+7195.511325201,3,100.333004,1.156787,2,42545152,12.597199,0.072683,1.161249,7.117647,2586.319103,0.000386,0.738719,2024-08-03 21:29:14.6632967 +0900 KST m=+7189.552568601,3523,100.072006,1.160825,1,45838336,,,,2587.479927,NaN,NaN,NaN,NaN,NaN,NaN,N

### Timestamp_n 간 차이 값 구하기

In [171]:
hop_df = preprocess_hop_log.process_timestamp(hop_df)

c:\Users\SJSJ\Desktop\wowsan_ai\preprocess\preprocess_hop_log.py:89: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  m_values_df = hop_df[timestamp_cols].applymap(extract_m_value)
c:\Users\SJSJ\Desktop\wowsan_ai\preprocess\preprocess_hop_log.py:94: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if pd.notna(row[i+1]):
c:\Users\SJSJ\Desktop\wowsan_ai\preprocess\preprocess_hop_log.py:95: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  next_value = row[i+1]
c:\Users\SJSJ\Desktop\wowsan_ai\preprocess\preprocess_hop_log.py:96: FutureWarning: Series.__getitem__ treating keys as positions is 

In [172]:
hop_df

,time,Node,level,msg,HopCount,QueueTime_0,Throughput_0,InterArrivalTime_0,QueueLength_0,Cpu_0,ServiceTime_0,BrokerId_0,Memory_0,AverageQueueTime_0,AverageThroughput_0,AverageServiceTime_0,ResponseTime_0,QueueTime_1,Throughput_1,InterArrivalTime_1,QueueLength_1,Cpu_1,ServiceTime_1,BrokerId_1,Memory_1,AverageQueueTime_1,AverageThroughput_1,AverageServiceTime_1,ResponseTime_1,QueueTime_2,Throughput_2,InterArrivalTime_2,QueueLength_2,Cpu_2,ServiceTime_2,BrokerId_2,Memory_2,AverageQueueTime_2,AverageThroughput_2,AverageServiceTime_2,ResponseTime_2,Timestamp_0,Timestamp_1,Timestamp_2
0,2024-08-03 19:29:35,localhost:50002,info,Advertisement(apple > 0.8346767073039755) ca036fac-2b7f-4d21-9e1f-81f1f4dbedb0,2,1.162724,0.860050,0.227600,3,100.110671,0.000000,2,23736320,1.162724,0.860050,0.000000,1.162724,0.903563,1.106730,0.001165,3,99.396370,0.000000,3,23515136,,,,0.903563,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.167725,0.000000,0.0
1,2024-08-03 19:29:36,localhost:50002,info,Advertisement(apple > 0.5348975819733137) 76ad8890-e29f-41cb-8e1d-7df43ca35e1b,2,2.070749,0.308502,0.284757,3,100.019933,1.170726,2,24195072,1.616736,0.358749,1.170726,3.241475,1.161399,0.861030,0.001046,1,99.684715,0.000000,1,23453696,,,,1.161399,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.070645,0.000000,0.0
2,2024-08-03 19:29:37,localhost:50002,info,Advertisement(apple > 0.5348975819733137) 25a1782f-2262-442b-80cd-c6baedb9f34a,2,3.224251,0.227966,0.284757,2,100.023972,1.162372,2,24453120,2.152574,0.301284,1.166549,4.386623,1.161399,0.861030,0.000000,0,100.851342,0.000000,4,23453696,,,,1.161399,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.274211,0.000000,0.0
3,2024-08-03 19:29:38,localhost:50002,info,Advertisement(apple > 0.5348975819733137) 9948d2fd-d7a6-4671-bfde-4677809e5177,2,3.726109,0.205060,0.284757,1,100.046152,1.150501,2,24461312,2.545958,0.269748,1.161200,4.876610,1.814963,0.367171,0.001165,2,99.399814,0.908564,3,24133632,,,,2.723527,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.730110,0.000000,0.0
4,2024-08-03 19:29:40,localhost:50002,info,Advertisement(apple > 0.5010479585914438) 4996a6de-461e-4014-94a2-83b7ed36aee1,3,4.365799,0.181202,0.284757,0,100.063807,1.152909,2,24477696,2.909926,0.245757,1.159127,5.518708,1.161434,0.429399,0.519194,2,100.808678,1.167401,4,24334336,,,,2.328835,1.161911,0.860651,0.000000,0,99.684467,0.000000,5.0,22827008,,,,1.161911,4.413758,1.162318,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6185,2024-08-03 21:29:19,localhost:50002,info,Advertisement(apple > 0.7830639144609146) f012e3a0-1ba8-450f-adb7-54a06446646d,2,4.847540,0.166560,1.160724,3,100.333072,1.156288,2,42545152,12.598271,0.072677,1.161250,6.003828,1508.480024,0.000662,0.914329,1662,100.194775,1.156824,4,45146112,,,,1509.636848,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.894499,0.000000,0.0
6186,2024-08-03 21:29:20,localhost:50002,info,Advertisement(apple > 0.9539755063328923) 569e0625-c121-42b0-9ee4-5c33d6978384,2,5.960860,0.140496,1.160930,3,100.333004,1.156787,2,42545152,12.597199,0.072683,1.161249,7.117647,2586.319103,0.000386,0.738719,3523,100.072006,1.160825,1,45838336,,,,2587.479927,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.958757,0.000000,0.0
6187,2024-08-03 21:29:21,localhost:50002,info,Advertisement(apple > 0.7014513757059698) 7d8a793f-b330-4f41-aff9-06d156eb36d5,2,5.631439,0.147302,1.160937,3,100.333167,1.157331,2,42545152,12.596073,0.072689,1.161248,6.788770,1140.013103,0.000876,0.760009,1542,100.102777,0.900550,3,43786240,,,,1140.913652,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.632440,0.000000,0.0
6188,2024-08-03 21:29:22,localhost:50002,info,Advertisement(apple > 0.8381583196357842) 7c3e075e-aec9-4ae7-8bc0-59dbd6cb4f56,2,4.968774,0.163210,1.160561,5,100.333323,1.158287,2,42545152,12.594841,0.072695,1.161248,6.127061,1140.381321,0.000876,0.759991,1544,100.102696,0.913727

In [173]:
# 폴더 없으면 생성
if not os.path.exists(path_mergeddf):
    os.makedirs(path_mergeddf)

hop_df.to_csv(path_mergeddf+'hop_df.csv', date_format='%Y-%m-%d %H:%M:%S', index=False)


## Tick Log 전처리

In [174]:
# json 파일을 열고 각 줄을 개별적으로 파싱
# path = '../raw_data/tickLogger/116/'
data = []
with open(path_ticklog + '50002_tick.json', 'r') as file:
    for line in file:
        data.append(json.loads(line))

# 데이터를 DataFrame으로 변환
tick_df_raw = pd.DataFrame(data)
type(tick_df_raw)

pandas.core.frame.DataFrame

In [175]:
tick_df_raw

,AverageQueueTime,AverageServiceTime,AverageThroughput,Bottleneck,Cpu,InterArrivalTime,Memory,MessageCount,QueueLength,QueueTime,ResponseTime,ServiceTime,Throughput,TotalArrival Time,level,msg,time
0,0.000000,0.000000,0.000000,False,101.996824,0.000000,21110784,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,info,Performance Metrics,2024-08-03 19:29:26
1,0.000000,0.000000,0.000000,False,101.068073,0.000000,23252992,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,info,Performance Metrics,2024-08-03 19:29:27
2,0.000000,0.000000,0.000000,False,100.738760,0.000000,23269376,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,info,Performance Metrics,2024-08-03 19:29:28
3,0.000000,0.000000,0.000000,False,100.015773,0.000000,23273472,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,info,Performance Metrics,2024-08-03 19:29:29
4,0.000000,0.000000,0.000000,False,100.240208,0.000000,23281664,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,info,Performance Metrics,2024-08-03 19:29:30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7194,12.598271,1.161249,0.072677,False,100.333191,1.160724,42545152,6189,2,4.847540,6.004327,1.156787,0.166547,7183.722602,info,Performance Metrics,2024-08-03 21:29:20
7195,12.597199,1.161248,0.072683,False,100.333274,1.160930,42545152,6190,2,5.960860,7.118190,1.157331,0.140485,7186.155249,info,Performance Metrics,2024-08-03 21:29:21
7196,12.596073,1.161248,0.072689,False,100.333343,1.160937,42545152,6191,2,5.631439,6.789727,1.158287,0.147281,7187.358022,info,Performance Metrics,2024-08-03 21:29:22
7197,12.594841,1.161247,0.072695,False,100.333426,1.160561,42545152,6194,4,4.968774,6.123563,1.154789,0.163304,7188.513336,info,Performance Metrics,2024-08-03 21:29:23


In [176]:
# merged_df = tick_df.set_index('time').combine_first(hop_df.set_index('time')).reset_index()

In [177]:
merged_df = merge_hop_tick.merge_dataframes(hop_df, tick_df_raw)

In [178]:
merged_df

,AverageQueueTime,AverageServiceTime,AverageThroughput,Bottleneck,Cpu_0,InterArrivalTime_0,Memory_0,MessageCount,QueueLength_0,QueueTime_0,ResponseTime_0,ServiceTime_0,Throughput_0,TotalArrival Time,level,msg,time,Node,HopCount,BrokerId_0,AverageQueueTime_0,AverageThroughput_0,AverageServiceTime_0,QueueTime_1,Throughput_1,InterArrivalTime_1,QueueLength_1,Cpu_1,ServiceTime_1,BrokerId_1,Memory_1,AverageQueueTime_1,AverageThroughput_1,AverageServiceTime_1,ResponseTime_1,QueueTime_2,Throughput_2,InterArrivalTime_2,QueueLength_2,Cpu_2,ServiceTime_2,BrokerId_2,Memory_2,AverageQueueTime_2,AverageThroughput_2,AverageServiceTime_2,ResponseTime_2,Timestamp_0,Timestamp_1,Timestamp_2
0,0.000000,0.000000,0.000000,False,101.996824,0.000000,21110784,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,info,Performance Metrics,2024-08-03 19:29:26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.000000,0.000000,0.000000,False,101.068073,0.000000,23252992,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,info,Performance Metrics,2024-08-03 19:29:27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.000000,0.000000,0.000000,False,100.73876,0.000000,23269376,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,info,Performance Metrics,2024-08-03 19:29:28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.000000,0.000000,0.000000,False,100.015773,0.000000,23273472,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,info,Performance Metrics,2024-08-03 19:29:29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.000000,0.000000,0.000000,False,100.240208,0.000000,23281664,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,info,Performance Metrics,2024-08-03 19:29:30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7194,12.598271,1.161249,0.072677,False,100.333004,1.160930,42545152,6189,3,5.960860,7.117647,1.156787,0.140496,7183.722602,info,Advertisement(apple > 0.9539755063328923) 569e0625-c121-42b0-9ee4-5c33d6978384,2024-08-03 21:29:20,localhost:50002,2.0,2.0,12.597199,0.072683,1.161249,2586.319103,0.000386,0.738719,3523,100.072006,1.160825,1.0,45838336,,,,2587.479927,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.958757,0.000000,0.0
7195,12.597199,1.161248,0.072683,False,100.333167,1.160937,42545152,6190,3,5.631439,6.788770,1.157331,0.147302,7186.155249,info,Advertisement(apple > 0.7014513757059698) 7d8a793f-b330-4f41-aff9-06d156eb36d5,2024-08-03 21:29:21,localhost:50002,2.0,2.0,12.596073,0.072689,1.161248,1140.013103,0.000876,0.760009,1542,100.102777,0.900550,3.0,43786240,,,,1140.913652,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.632440,0.000000,0.0
7196,12.596073,1.161248,0.072689,False,100.333323,1.160561,42545152,6191,5,4.968774,6.127061,1.158287,0.163210,7187.358022,info,Advertisement(apple > 0.8381583196357842) 7c3e075e-aec9-4ae7-8bc0-59dbd6cb4f56,2024-08-03 21:29:22,localhost:50002,2.0,2.0,12.594841,0.072695,1.161248,1140.381321,0.000876,0.759991,1544,100.102696,0.913727,3.0,43786240,,,,1141.295047,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.972775,0.000000,0.0
7197,12.594841,1.161247,0.072695,False,100.333426,1.160561,42545152,6194,4,4.968774,6.123563,1.154789,0.163304,7188.513336,info,Performance Metrics,2024-08-03 21:29:23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [179]:
# Bottleneck 컬럼의 False는 0으로, True는 1로 변환
merged_df['Bottleneck'] = merged_df['Bottleneck'].astype('float64')


In [180]:
# merged_df 컬럼 확인
merged_df.columns
# len(merged_df.columns)

Index(['AverageQueueTime', 'AverageServiceTime', 'AverageThroughput',
       'Bottleneck', 'Cpu_0', 'InterArrivalTime_0', 'Memory_0', 'MessageCount',
       'QueueLength_0', 'QueueTime_0', 'ResponseTime_0', 'ServiceTime_0',
       'Throughput_0', 'TotalArrival Time', 'level', 'msg', 'time', 'Node',
       'HopCount', 'BrokerId_0', 'AverageQueueTime_0', 'AverageThroughput_0',
       'AverageServiceTime_0', 'QueueTime_1', 'Throughput_1',
       'InterArrivalTime_1', 'QueueLength_1', 'Cpu_1', 'ServiceTime_1',
       'BrokerId_1', 'Memory_1', 'AverageQueueTime_1', 'AverageThroughput_1',
       'AverageServiceTime_1', 'ResponseTime_1', 'QueueTime_2', 'Throughput_2',
       'InterArrivalTime_2', 'QueueLength_2', 'Cpu_2', 'ServiceTime_2',
       'BrokerId_2', 'Memory_2', 'AverageQueueTime_2', 'AverageThroughput_2',
       'AverageServiceTime_2', 'ResponseTime_2', 'Timestamp_0', 'Timestamp_1',
       'Timestamp_2'],
      dtype='object')

### 학습에 필요한 컬럼만 남긴다

In [181]:
# 학습에 불필요한 컬럼 제거
# 불필요한 컬럼 리스트
drop_columns = [
    'level',
    'msg',
    'Node',
    # 'TotalArrivalTime',
    'MessageCount',
    # 'Timestamp'
]

# 불필요한 컬럼 제거
merged_df.drop(columns=drop_columns, inplace=True)

In [182]:
merged_df = merged_df.rename(columns={'time': 'date'})

In [183]:
merged_df  

,AverageQueueTime,AverageServiceTime,AverageThroughput,Bottleneck,Cpu_0,InterArrivalTime_0,Memory_0,QueueLength_0,QueueTime_0,ResponseTime_0,ServiceTime_0,Throughput_0,TotalArrival Time,date,HopCount,BrokerId_0,AverageQueueTime_0,AverageThroughput_0,AverageServiceTime_0,QueueTime_1,Throughput_1,InterArrivalTime_1,QueueLength_1,Cpu_1,ServiceTime_1,BrokerId_1,Memory_1,AverageQueueTime_1,AverageThroughput_1,AverageServiceTime_1,ResponseTime_1,QueueTime_2,Throughput_2,InterArrivalTime_2,QueueLength_2,Cpu_2,ServiceTime_2,BrokerId_2,Memory_2,AverageQueueTime_2,AverageThroughput_2,AverageServiceTime_2,ResponseTime_2,Timestamp_0,Timestamp_1,Timestamp_2
0,0.000000,0.000000,0.000000,0.0,101.996824,0.000000,21110784,0,0.000000,0.000000,0.000000,0.000000,0.000000,2024-08-03 19:29:26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.000000,0.000000,0.000000,0.0,101.068073,0.000000,23252992,0,0.000000,0.000000,0.000000,0.000000,0.000000,2024-08-03 19:29:27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.000000,0.000000,0.000000,0.0,100.73876,0.000000,23269376,0,0.000000,0.000000,0.000000,0.000000,0.000000,2024-08-03 19:29:28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.000000,0.000000,0.000000,0.0,100.015773,0.000000,23273472,0,0.000000,0.000000,0.000000,0.000000,0.000000,2024-08-03 19:29:29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.000000,0.000000,0.000000,0.0,100.240208,0.000000,23281664,0,0.000000,0.000000,0.000000,0.000000,0.000000,2024-08-03 19:29:30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7194,12.598271,1.161249,0.072677,0.0,100.333004,1.160930,42545152,3,5.960860,7.117647,1.156787,0.140496,7183.722602,2024-08-03 21:29:20,2.0,2.0,12.597199,0.072683,1.161249,2586.319103,0.000386,0.738719,3523,100.072006,1.160825,1.0,45838336,,,,2587.479927,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.958757,0.000000,0.0
7195,12.597199,1.161248,0.072683,0.0,100.333167,1.160937,42545152,3,5.631439,6.788770,1.157331,0.147302,7186.155249,2024-08-03 21:29:21,2.0,2.0,12.596073,0.072689,1.161248,1140.013103,0.000876,0.760009,1542,100.102777,0.900550,3.0,43786240,,,,1140.913652,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.632440,0.000000,0.0
7196,12.596073,1.161248,0.072689,0.0,100.333323,1.160561,42545152,5,4.968774,6.127061,1.158287,0.163210,7187.358022,2024-08-03 21:29:22,2.0,2.0,12.594841,0.072695,1.161248,1140.381321,0.000876,0.759991,1544,100.102696,0.913727,3.0,43786240,,,,1141.295047,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.972775,0.000000,0.0
7197,12.594841,1.161247,0.072695,0.0,100.333426,1.160561,42545152,4,4.968774,6.123563,1.154789,0.163304,7188.513336,2024-08-03 21:29:23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [184]:
# # Timestamp_0 컬럼 제거
# merged_df.drop(columns=['Timestamp_0'], inplace=True)

# # 1500 행 이후의 데이터를 제거
# merged_df = merged_df[:1500]

# 모든 값이 NaN인 컬럼 이름 출력 후 해당 컬럼 제거
print(merged_df.columns[merged_df.isna().all()].tolist())
merged_df.drop(columns=merged_df.columns[merged_df.isna().all()].tolist(), inplace=True)


[]


In [185]:
# merge_df 의 컬럼 출력
merged_df.columns

Index(['AverageQueueTime', 'AverageServiceTime', 'AverageThroughput',
       'Bottleneck', 'Cpu_0', 'InterArrivalTime_0', 'Memory_0',
       'QueueLength_0', 'QueueTime_0', 'ResponseTime_0', 'ServiceTime_0',
       'Throughput_0', 'TotalArrival Time', 'date', 'HopCount', 'BrokerId_0',
       'AverageQueueTime_0', 'AverageThroughput_0', 'AverageServiceTime_0',
       'QueueTime_1', 'Throughput_1', 'InterArrivalTime_1', 'QueueLength_1',
       'Cpu_1', 'ServiceTime_1', 'BrokerId_1', 'Memory_1',
       'AverageQueueTime_1', 'AverageThroughput_1', 'AverageServiceTime_1',
       'ResponseTime_1', 'QueueTime_2', 'Throughput_2', 'InterArrivalTime_2',
       'QueueLength_2', 'Cpu_2', 'ServiceTime_2', 'BrokerId_2', 'Memory_2',
       'AverageQueueTime_2', 'AverageThroughput_2', 'AverageServiceTime_2',
       'ResponseTime_2', 'Timestamp_0', 'Timestamp_1', 'Timestamp_2'],
      dtype='object')

In [186]:
new_order = ['date', 'Bottleneck', 'HopCount', 
             'Timestamp_0', 'BrokerId_0', 'Cpu_0', 'Memory_0', 'InterArrivalTime_0', 'QueueLength_0', 'QueueTime_0', 'ServiceTime_0', 'ResponseTime_0', 'Throughput_0',
             'Timestamp_1',  'BrokerId_1', 'Cpu_1', 'Memory_1', 'InterArrivalTime_1', 'QueueLength_1',  'QueueTime_1', 'ServiceTime_1', 'ResponseTime_1', 'Throughput_1',
             'Timestamp_2', 'BrokerId_2', 'Cpu_2', 'Memory_2', 'InterArrivalTime_2', 'QueueLength_2', 'QueueTime_2', 'ServiceTime_2', 'ResponseTime_2', 'Throughput_2']

# 새로운 순서로 데이터프레임을 재정렬합니다
merged_df = merged_df[new_order]

In [187]:
drop_columns = [ 
    'Bottleneck',
#     'BrokerId_0',
#     'BrokerId_1',
#     'BrokerId_2'
]

# # 불필요한 컬럼 제거
merged_df.drop(columns=drop_columns, inplace=True)

C:\Users\SJSJ\AppData\Local\Temp\ipykernel_25672\2104203700.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df.drop(columns=drop_columns, inplace=True)


In [188]:
# return all unique values of merged_df columns
for col in merged_df.columns:
    print(col)
    print(merged_df[col].unique())
    print()


date
['2024-08-03 19:29:26' '2024-08-03 19:29:27' '2024-08-03 19:29:28' ...
 '2024-08-03 21:29:22' '2024-08-03 21:29:23' '2024-08-03 21:29:24']

HopCount
[nan  2.  3.]

Timestamp_0
[      nan 1.1677253 2.0706454 ... 5.6324397 4.9727747 3.7393569]

BrokerId_0
[nan  2.]

Cpu_0
[101.99682422688088 101.06807269139205 100.7387596761018 ... '100.333323'
 100.33342594888359 '100.333255']

Memory_0
[21110784 23252992 23269376 ... 42541056 '42545152' 42545152]

InterArrivalTime_0
['0.000000' '0.227600' '0.284757' ... '1.160930' '1.160937' '1.160561']

QueueLength_0
[0 '3' '2' '1' '0' 1 '4' '5' 4 3 '6' 5 '7' '8' 7 '9' 8 6 '10' 9 '11' 10
 '12' '13' 12 11 '14' 13 '15' 14 '16' 15 '17' '19' '18' 17 '20' 18 16 2]

QueueTime_0
['0.000000' '1.162724' '2.070749' ... '5.631439' '4.968774' '3.693398']

ServiceTime_0
['0.000000' '1.170726' '1.162372' ... '1.153780' '1.158287' '1.154789']

ResponseTime_0
['0.000000' '1.162724' '3.241475' ... '6.127061' '6.123563' '4.848187']

Throughput_0
['0.000000' '0.860

In [189]:
# NaN 값을 0으로 대체
merged_df.fillna(0, inplace=True)

C:\Users\SJSJ\AppData\Local\Temp\ipykernel_25672\1254312440.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df.fillna(0, inplace=True)


In [190]:
# fill_nan.fill_nan(merged_df)

In [191]:
merged_df

,date,HopCount,Timestamp_0,BrokerId_0,Cpu_0,Memory_0,InterArrivalTime_0,QueueLength_0,QueueTime_0,ServiceTime_0,ResponseTime_0,Throughput_0,Timestamp_1,BrokerId_1,Cpu_1,Memory_1,InterArrivalTime_1,QueueLength_1,QueueTime_1,ServiceTime_1,ResponseTime_1,Throughput_1,Timestamp_2,BrokerId_2,Cpu_2,Memory_2,InterArrivalTime_2,QueueLength_2,QueueTime_2,ServiceTime_2,ResponseTime_2,Throughput_2
0,2024-08-03 19:29:26,0.0,0.000000,0.0,101.996824,21110784,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0
1,2024-08-03 19:29:27,0.0,0.000000,0.0,101.068073,23252992,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0
2,2024-08-03 19:29:28,0.0,0.000000,0.0,100.73876,23269376,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0
3,2024-08-03 19:29:29,0.0,0.000000,0.0,100.015773,23273472,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0
4,2024-08-03 19:29:30,0.0,0.000000,0.0,100.240208,23281664,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7194,2024-08-03 21:29:20,2.0,5.958757,2.0,100.333004,42545152,1.160930,3,5.960860,1.156787,7.117647,0.140496,0.000000,1.0,100.072006,45838336,0.738719,3523,2586.319103,1.160825,2587.479927,0.000386,0.0,0.0,0,0,0,0,0,0,0,0
7195,2024-08-03 21:29:21,2.0,5.632440,2.0,100.333167,42545152,1.160937,3,5.631439,1.157331,6.788770,0.147302,0.000000,3.0,100.102777,43786240,0.760009,1542,1140.013103,0.900550,1140.913652,0.000876,0.0,0.0,0,0,0,0,0,0,0,0
7196,2024-08-03 21:29:22,2.0,4.972775,2.0,100.333323,42545152,1.160561,5,4.968774,1.158287,6.127061,0.163210,0.000000,3.0,100.102696,43786240,0.759991,1544,1140.381321,0.913727,1141.295047,0.000876,0.0,0.0,0,0,0,0,0,0,0,0
7197,2024-08-03 21:29:23,0.0,0.000000,0.0,100.333426,42545152,1.160561,4,4.968774,1.154789,6.123563,0.163304,0.000000,0.0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0


In [192]:
# 각 컬럼별로 unique 값이 하나인 컬럼 삭제
for col in merged_df.columns:
    if len(merged_df[col].unique()) == 1:
        print(col)
        merged_df = merged_df.drop(col, axis=1)

Timestamp_2


In [193]:
# save to csv
# 디렉토리가 없으면 생성
if not os.path.exists(path_mergeddf):
    os.makedirs(path_mergeddf)

merged_df.to_csv(path_mergeddf+'merged_df.csv', date_format='%Y-%m-%d %H:%M:%S', index=False)


In [194]:
merged_df

,date,HopCount,Timestamp_0,BrokerId_0,Cpu_0,Memory_0,InterArrivalTime_0,QueueLength_0,QueueTime_0,ServiceTime_0,ResponseTime_0,Throughput_0,Timestamp_1,BrokerId_1,Cpu_1,Memory_1,InterArrivalTime_1,QueueLength_1,QueueTime_1,ServiceTime_1,ResponseTime_1,Throughput_1,BrokerId_2,Cpu_2,Memory_2,InterArrivalTime_2,QueueLength_2,QueueTime_2,ServiceTime_2,ResponseTime_2,Throughput_2
0,2024-08-03 19:29:26,0.0,0.000000,0.0,101.996824,21110784,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0
1,2024-08-03 19:29:27,0.0,0.000000,0.0,101.068073,23252992,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0
2,2024-08-03 19:29:28,0.0,0.000000,0.0,100.73876,23269376,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0
3,2024-08-03 19:29:29,0.0,0.000000,0.0,100.015773,23273472,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0
4,2024-08-03 19:29:30,0.0,0.000000,0.0,100.240208,23281664,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7194,2024-08-03 21:29:20,2.0,5.958757,2.0,100.333004,42545152,1.160930,3,5.960860,1.156787,7.117647,0.140496,0.000000,1.0,100.072006,45838336,0.738719,3523,2586.319103,1.160825,2587.479927,0.000386,0.0,0,0,0,0,0,0,0,0
7195,2024-08-03 21:29:21,2.0,5.632440,2.0,100.333167,42545152,1.160937,3,5.631439,1.157331,6.788770,0.147302,0.000000,3.0,100.102777,43786240,0.760009,1542,1140.013103,0.900550,1140.913652,0.000876,0.0,0,0,0,0,0,0,0,0
7196,2024-08-03 21:29:22,2.0,4.972775,2.0,100.333323,42545152,1.160561,5,4.968774,1.158287,6.127061,0.163210,0.000000,3.0,100.102696,43786240,0.759991,1544,1140.381321,0.913727,1141.295047,0.000876,0.0,0,0,0,0,0,0,0,0
7197,2024-08-03 21:29:23,0.0,0.000000,0.0,100.333426,42545152,1.160561,4,4.968774,1.154789,6.123563,0.163304,0.000000,0.0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0


In [195]:
# 컬럼별 datatype 확인
merged_df.dtypes

date                   object
HopCount              float64
Timestamp_0           float64
BrokerId_0            float64
Cpu_0                  object
Memory_0               object
InterArrivalTime_0     object
QueueLength_0          object
QueueTime_0            object
ServiceTime_0          object
ResponseTime_0         object
Throughput_0           object
Timestamp_1           float64
BrokerId_1            float64
Cpu_1                  object
Memory_1               object
InterArrivalTime_1     object
QueueLength_1          object
QueueTime_1            object
ServiceTime_1          object
ResponseTime_1         object
Throughput_1           object
BrokerId_2            float64
Cpu_2                  object
Memory_2               object
InterArrivalTime_2     object
QueueLength_2          object
QueueTime_2            object
ServiceTime_2          object
ResponseTime_2         object
Throughput_2           object
dtype: object